# Демонстрация использования класса `OscillogramActivityFilter`

Этот ноутбук показывает, как использовать класс `OscillogramActivityFilter` из `analysis.activity_filter` для фильтрации "пустых" или неактивных осциллограмм. Фильтр анализирует первую гармонику (H1) выбранных каналов и определяет активность на основе заданных порогов для дельты, стандартного отклонения и максимального значения H1.

Поддерживается два режима анализа для каждого канала:
1.  С использованием нормализованных данных (если предоставлен `OscillogramNormalizer` и коэффициенты, и опция `use_norm_osc` в конфигурации включена).
2.  Анализ "сырых" данных с относительными порогами (если нормализация не используется или не удалась). В этом режиме сигнал сначала проверяется на "чистоту" начального участка.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json 
import sys
import csv # For sample norm_coef.csv

# Настройка путей для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from core.oscillogram import Oscillogram
from normalization.normalization import OscillogramNormalizer 
from analysis.activity_filter import OscillogramActivityFilter, ChannelType 

# --- Создание временных директорий ---
base_sample_dir_af = "temp_sample_data_activity_filter"
source_dir_af = os.path.join(base_sample_dir_af, "source_oscillograms_af")
output_dir_af = os.path.join(base_sample_dir_af, "activity_filter_output")

if os.path.exists(base_sample_dir_af):
    shutil.rmtree(base_sample_dir_af)
os.makedirs(source_dir_af)
os.makedirs(output_dir_af)

# --- Вспомогательная функция для создания CFG/DAT ---
def create_af_sample_comtrade(path, name, analog_ch_details, freq=50.0, samp_rate=1000.0, dat_rows=200):
    cfg_content = f"SampleStation,AF_Device,1999\n"
    cfg_content += f"{len(analog_ch_details)},{len(analog_ch_details)}A,0D\n"
    ch_idx = 1
    for ch_name, _, _ in analog_ch_details: 
        cfg_content += f"{ch_idx},{ch_name},A,,V,1.0,0.0,0,-32767,32767,1,1,P\n" 
        ch_idx += 1
    cfg_content += f"{freq}\n1\n{samp_rate},{dat_rows}\n"
    cfg_content += "01/01/2023,00:00:00.000000\n01/01/2023,00:00:00.000000\nASCII\n1.0\n"
    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f: f.write(cfg_content)

    dat_content = ""
    for i in range(dat_rows):
        dat_content += f"{i+1},{int(i*(1000000/samp_rate))}"
        for _, data_series, _ in analog_ch_details: 
            dat_content += f",{data_series[i % len(data_series)]:.6f}"
        dat_content += "\n"
    with open(os.path.join(path, f"{name}.dat"), "w", encoding="utf-8") as f: f.write(dat_content)

# --- Создание файлов осциллограмм ---
time_pts_af = np.linspace(0, 2*np.pi*10, 200) 

active_signal_data = np.concatenate([np.sin(time_pts_af[:100])*0.2, np.sin(time_pts_af[100:])*1.0])
create_af_sample_comtrade(source_dir_af, "osc_af_active", [
    ("I | CurrentChannel1", active_signal_data, True)
])

empty_signal_data = np.random.rand(200) * 0.001 
create_af_sample_comtrade(source_dir_af, "osc_af_empty_noise", [
    ("U | VoltageChannelEmpty", empty_signal_data, False)
])

stable_signal_data = np.sin(time_pts_af) * 0.5
create_af_sample_comtrade(source_dir_af, "osc_af_empty_stable", [
    ("I | CurrentChannelStable", stable_signal_data, False)
])

active_for_norm_data = np.concatenate([np.sin(time_pts_af[:100])*2000, np.sin(time_pts_af[100:])*10000]) 
create_af_sample_comtrade(source_dir_af, "osc_af_active_fornorm", [
    ("U | VoltageForNorm", active_for_norm_data, True)
])

print(f"Создана директория с примерами: {source_dir_af}")

activity_filter_config = {
    'channels_to_analyze_patterns': ['i |', 'u |'],
    'current_channel_id_patterns': ['i |'],
    'voltage_channel_id_patterns': ['u |'],
    'use_norm_osc': True, 
    'norm_yes_phrase': 'YES', # What to look for in 'norm' column of norm_coef.csv
    'thresholds_current_normalized': {'delta': 0.05, 'std_dev': 0.02, 'max_abs': 0.01},
    'thresholds_voltage_normalized': {'delta': 0.03, 'std_dev': 0.015, 'max_abs': 0.02},
    'raw_signal_analysis': {
        'initial_window_check_periods': 1, 
        'h1_vs_hx_ratio_threshold_U': 5, 
        'h1_vs_hx_ratio_threshold_I': 3, 
        'min_initial_h1_amplitude_for_rel_norm': 0.01, # Min H1 peak for raw signal to be considered for relative norm
        'thresholds_raw_current_relative': {'delta': 0.2, 'std_dev': 0.1, 'max_abs': 0.1}, # max_abs is optional here
        'thresholds_raw_voltage_relative': {'delta': 0.1, 'std_dev': 0.05, 'max_abs': 0.1}
    },
    'verbose': False 
}

physical_phase_nominal_U_rms = 10000.0 / np.sqrt(3)
physical_phase_nominal_I_rms = 20.0 # Example primary current

norm_data_af = [
    ["name", "norm", "1Ub_base", "1Ip_base"], 
    ["osc_af_active", "YES", physical_phase_nominal_U_rms/3.0, physical_phase_nominal_I_rms/20.0], 
    ["osc_af_empty_noise", "YES", physical_phase_nominal_U_rms/3.0, physical_phase_nominal_I_rms/20.0],
    ["osc_af_empty_stable", "YES", physical_phase_nominal_U_rms/3.0, physical_phase_nominal_I_rms/20.0],
    ["osc_af_active_fornorm", "YES", physical_phase_nominal_U_rms/3.0, physical_phase_nominal_I_rms/20.0]
]
sample_norm_coef_path_af = os.path.join(output_dir_af, "sample_norm_coef_af.csv")
with open(sample_norm_coef_path_af, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(norm_data_af)
print(f"Создан файл коэффициентов нормализации: {sample_norm_coef_path_af}")

## 1. Инициализация `OscillogramActivityFilter`

In [ ]:
normalizer_af = None
if activity_filter_config.get('use_norm_osc', False):
    normalizer_af = OscillogramNormalizer(norm_coef_file_path=sample_norm_coef_path_af, is_print_message=True)

activity_filter = OscillogramActivityFilter(
    config=activity_filter_config,
    normalizer=normalizer_af
)

print("OscillogramActivityFilter инициализирован.")
if normalizer_af:
    print("Будет использоваться нормализация (если разрешено для файла в norm_coef.csv).")
else:
    print("Нормализация не будет использоваться.")

## 2. Фильтрация директории (`filter_directory`)
Метод `filter_directory` обрабатывает все осциллограммы в указанной директории и сохраняет список "непустых" (активных) файлов в CSV.

In [ ]:
active_files_csv_path = os.path.join(output_dir_af, "active_oscillograms.csv")

print(f"\nЗапуск фильтрации директории: {source_dir_af}")
activity_filter.verbose = True 
activity_filter.filter_directory(
    source_dir=source_dir_af,
    output_csv_path=active_files_csv_path
)
activity_filter.verbose = False 

if os.path.exists(active_files_csv_path):
    print(f"\nСодержимое отчета ({active_files_csv_path}):")
    try: 
        from IPython.display import display
        display(pd.read_csv(active_files_csv_path))
    except ImportError:
        print(pd.read_csv(active_files_csv_path))
else:
    print(f"Отчет о непустых файлах не был создан.")

## 3. Проверка активности отдельного файла (`check_activity`)
Можно проверить активность конкретной осциллограммы.

In [ ]:
osc_active_path = os.path.join(source_dir_af, "osc_af_active.cfg")
osc_empty_path = os.path.join(source_dir_af, "osc_af_empty_stable.cfg")
osc_active_norm_path = os.path.join(source_dir_af, "osc_af_active_fornorm.cfg")

activity_filter.verbose = True 
try:
    print(f"\nПроверка файла: {os.path.basename(osc_active_path)}")
    osc1 = Oscillogram(osc_active_path)
    is_active1 = activity_filter.check_activity(osc1)
    print(f"Результат для {os.path.basename(osc_active_path)}: {'Активен' if is_active1 else 'Пустой'}")

    print(f"\nПроверка файла: {os.path.basename(osc_empty_path)}")
    osc2 = Oscillogram(osc_empty_path)
    is_active2 = activity_filter.check_activity(osc2)
    print(f"Результат для {os.path.basename(osc_empty_path)}: {'Активен' if is_active2 else 'Пустой'}")
    
    print(f"\nПроверка файла (требует нормализации): {os.path.basename(osc_active_norm_path)}")
    osc3 = Oscillogram(osc_active_norm_path)
    is_active3 = activity_filter.check_activity(osc3)
    print(f"Результат для {os.path.basename(osc_active_norm_path)}: {'Активен' if is_active3 else 'Пустой'}")

except Exception as e:
    print(f"Ошибка при проверке отдельных файлов: {e}")
activity_filter.verbose = False

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_sample_dir_af):
        shutil.rmtree(base_sample_dir_af)
        print(f"\nВременная директория {base_sample_dir_af} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_sample_dir_af}: {e}")